# **📚 BookVerse: The Library Navigator**
Welcome to BookVerse, an interactive Library Management System that helps you navigate a curated collection of self-help and fiction books. Built with Python, this system allows users to manage book inventories, borrow and return books, set up reservations, receive overdue notifications, and get personalized book recommendations.

Whether you're a student looking for insightful reads or an admin managing the collection, BookVerse offers a user-friendly, console-based experience designed for libraries focusing on classic and modern literary treasures.

**Features:**
- Inventory Management: Add, remove, search, and reserve books.
- Borrowing and Returning: Track borrowed books with due dates and overdue alerts.
- User Roles: Admin access to advanced management features.
- Personalized Recommendations: Suggested readings based on your recent choices.
- Statistics Dashboard: Insights into the most popular books and active users.

**Get ready to explore, organize, and manage a universe of books with ease!**

In [1]:
from datetime import datetime, timedelta

## Sample Data for Books, Users, and Transactions

In [2]:
books = [
    {"book_id": 201, "title": "Atomic Habits", "author": "James Clear", "genre": "Self-Help", "copies_available": 2, "status": "Available", "reservations": []},
    {"book_id": 202, "title": "To Kill a Mockingbird", "author": "Harper Lee", "genre": "Fiction", "copies_available": 3, "status": "Available", "reservations": []},
    {"book_id": 203, "title": "The Power of Now", "author": "Eckhart Tolle", "genre": "Self-Help", "copies_available": 1, "status": "Borrowed", "reservations": []},
    {"book_id": 204, "title": "Pride and Prejudice", "author": "Jane Austen", "genre": "Fiction", "copies_available": 4, "status": "Available", "reservations": []},
    {"book_id": 205, "title": "The 7 Habits of Highly Effective People", "author": "Stephen R. Covey", "genre": "Self-Help", "copies_available": 2, "status": "Available", "reservations": []}
]

users = {
    301: {"user_id": 301, "name": "Evelyn", "role": "Student", "borrow_history": [], "overdue_books": [], "password": "evelyn123"},
    302: {"user_id": 302, "name": "Henry", "role": "Admin", "borrow_history": [], "overdue_books": [], "password": "adminpass"},
    303: {"user_id": 303, "name": "Liam", "role": "Student", "borrow_history": [], "overdue_books": [], "password": "liam456"},
    304: {"user_id": 304, "name": "Sophia", "role": "Student", "borrow_history": [], "overdue_books": [], "password": "sophia789"}
}

transactions = [
    {"transaction_id": 1, "user_id": 301, "book_id": 203, "borrow_date": "2024-10-01", "due_date": "2024-10-15", "return_date": None, "overdue_status": "Overdue"},
    {"transaction_id": 2, "user_id": 302, "book_id": 201, "borrow_date": "2024-10-05", "due_date": "2024-10-19", "return_date": None, "overdue_status": "Not Overdue"}
]

In [3]:
# Helper function to find a book by its ID
def find_book_by_id(book_id):
    for book in books:
        if book["book_id"] == book_id:
            return book
    return None

In [4]:
# Helper function to find a user by their ID
def find_user_by_id(user_id):
    if user_id in users:
        return users[user_id]
    return None

In [5]:
# Function for user login
def user_login():
    print("=== Login ===")
    user_id = int(input("Enter your User ID: ").strip())
    password = input("Enter your password: ").strip()

    user = find_user_by_id(user_id)
    if user and user["password"] == password:
        print(f"Welcome, {user['name']} ({user['role']})!")
        return user
    else:
        print("Error: Invalid user ID or password.")
        return None

In [6]:
# Helper function to check if user is an admin
def is_admin(user):
    return user["role"].lower() == "admin"

## Book Inventory Management: Add, Remove, and Search for Books


In [7]:
# Function to add a new book to the inventory
def add_book(book_id, title, author, genre, copies_available):
    if find_book_by_id(book_id):
        print(f"Error: A book with ID {book_id} already exists.")
        return
    books.append({
        "book_id": book_id,
        "title": title,
        "author": author,
        "genre": genre,
        "copies_available": copies_available,
        "status": "Available" if copies_available > 0 else "Borrowed",
        "reservations": []
    })
    print(f"Book '{title}' added to the inventory.")

In [8]:
# Function to remove a book from the inventory
def remove_book(book_id):
    book = find_book_by_id(book_id)
    if book:
        books.remove(book)
        print(f"Book '{book['title']}' removed from the inventory.")
    else:
        print(f"Error: Book with ID {book_id} not found.")

In [9]:
# Function to search for a book by title
def search_book(title):
    found = False
    for book in books:
        if book["title"].lower() == title.lower():
            print(f"Book found: {book['title']} by {book['author']}, Genre: {book['genre']}, Status: {book['status']}")
            found = True
    if not found:
        print("Book not found.")

## Reservation System: Reserve a Book if All Copies Are Borrowed

In [10]:
def reserve_book(user_id, book_id):
    book = find_book_by_id(book_id)
    if not book:
        print("Error: Book not found.")
        return

    if book["copies_available"] == 0:
        book["reservations"].append(user_id)
        print(f"Book '{book['title']}' reserved successfully. You will be notified when it becomes available.")
    else:
        print("Error: Book is currently available. You can borrow it directly.")

## Borrow and Return System

In [11]:
# Function to borrow a book
def borrow_book(user_id, book_id):
    user = find_user_by_id(user_id)
    book = find_book_by_id(book_id)

    if not user:
        print("Error: User not found.")
        return

    if not book:
        print("Error: Book not found.")
        return

    if book["copies_available"] > 0:
        book["copies_available"] -= 1
        book["status"] = "Borrowed" if book["copies_available"] == 0 else "Available"

        borrow_date = datetime.now()
        due_date = borrow_date + timedelta(days=14)  # 2-week borrowing period

        transactions.append({
            "transaction_id": len(transactions) + 1,
            "user_id": user_id,
            "book_id": book_id,
            "borrow_date": borrow_date.strftime('%Y-%m-%d'),
            "due_date": due_date.strftime('%Y-%m-%d'),
            "return_date": None,
            "overdue_status": "Not Overdue"
        })

        user["borrow_history"].append(book["title"])
        print(f"Book '{book['title']}' borrowed by {user['name']}. Due date: {due_date.strftime('%Y-%m-%d')}")
    else:
        print(f"Error: Book '{book['title']}' is currently unavailable.")

In [12]:
# Function to return a book
def return_book(user_id, book_id):
    user = find_user_by_id(user_id)
    book = find_book_by_id(book_id)

    if not user:
        print("Error: User not found.")
        return

    if not book:
        print("Error: Book not found.")
        return

    transaction_found = False
    for transaction in transactions:
        if transaction["user_id"] == user_id and transaction["book_id"] == book_id and transaction["return_date"] is None:
            return_date = datetime.now()
            transaction["return_date"] = return_date.strftime('%Y-%m-%d')

            due_date = datetime.strptime(transaction["due_date"], '%Y-%m-%d')
            if return_date > due_date:
                transaction["overdue_status"] = "Overdue"
                print(f"Book '{book['title']}' returned late by {user['name']}.")
            else:
                print(f"Book '{book['title']}' returned on time by {user['name']}.")

            book["copies_available"] += 1
            book["status"] = "Available"
            user["borrow_history"].remove(book["title"])
            transaction_found = True
            break
    if not transaction_found:
        print(f"Error: No active transaction found for this book by user {user['name']}.")

# Overdue Notification Function


In [13]:
# Function to check and notify users of overdue books
def send_overdue_notifications():
    print("Sending overdue notifications...")
    overdue_found = False
    for transaction in transactions:
        if transaction["return_date"] is None:
            due_date = datetime.strptime(transaction["due_date"], '%Y-%m-%d')
            if datetime.now() > due_date:
                user = find_user_by_id(transaction["user_id"])
                book = find_book_by_id(transaction["book_id"])
                print(f"Notification: Book '{book['title']}' is overdue for {user['name']}. Please return it.")
                overdue_found = True
    if not overdue_found:
        print("No overdue books at the moment.")

## Book Recommendations Based on Genre

In [14]:
def recommend_books(user_id):
    user = find_user_by_id(user_id)
    if not user or not user["borrow_history"]:
        print("No borrow history available for recommendations.")
        return

    last_borrowed_title = user["borrow_history"][-1]
    last_borrowed_book = next((book for book in books if book["title"] == last_borrowed_title), None)

    if last_borrowed_book:
        genre = last_borrowed_book["genre"]
        print(f"Recommended books in genre '{genre}':")
        for book in books:
            if book["genre"] == genre and book["title"] != last_borrowed_title:
                print(f"- {book['title']} by {book['author']} ({'Available' if book['copies_available'] > 0 else 'Borrowed'})")
    else:
        print("No suitable recommendations found.")

## Library Statistics

In [ ]:
def most_borrowed_books():
    borrow_count = {book["title"]: 0 for book in books}
    for transaction in transactions:
        book = find_book_by_id(transaction["book_id"])
        if book:
            borrow_count[book["title"]] += 1

    print("Most Borrowed Books:")
    sorted_books = sorted(borrow_count.items(), key=lambda x: x[1], reverse=True)
    for title, count in sorted_books[:5]:  # Top 5 most borrowed books
        print(f"{title}: {count} times")

def most_active_users():
    user_activity = {user["name"]: len(user["borrow_history"]) for user in users.values()}

    print("Most Active Users:")
    sorted_users = sorted(user_activity.items(), key=lambda x: x[1], reverse=True)
    for name, count in sorted_users[:5]:  # Top 5 most active users
        print(f"{name}: {count} books borrowed")

# User Interaction: Search and Manage Books via Console

In [16]:
# Main function to interact with the user
def user_interaction():
    print("Welcome to the Library Management System")
    print("1. Register as a new user")
    print("2. Login to the system")

    choice = input("Select an option: ")
    if choice == '1':
        register_user()
        return  # Exit after registration to start the login process again
    elif choice != '2':
        print("Invalid option.")
        return

    user = user_login()
    if not user:
        return  # Exit if login fails

    while True:
        print("\n--- Main Menu ---")
        print("1. Search Book")
        print("2. Borrow Book")
        print("3. Return Book")
        print("4. Add Book (Admin)")
        print("5. Remove Book (Admin)")
        print("6. Send Overdue Notifications")
        print("7. Reserve Book")
        print("8. Book Recommendations")
        print("9. View Library Statistics (Admin)")
        print("10. Exit")

        choice = input("Select an option: ")

        if choice == '1':
            title = input("Enter book title to search: ").strip()
            search_book(title)
        elif choice == '2':
            book_id = int(input("Enter the book ID to borrow: ").strip())
            borrow_book(user["user_id"], book_id)
        elif choice == '3':
            book_id = int(input("Enter the book ID to return: ").strip())
            return_book(user["user_id"], book_id)
        elif choice == '4' and is_admin(user):
            book_id = int(input("Enter new book ID: ").strip())
            title = input("Enter book title: ").strip()
            author = input("Enter book author: ").strip()
            genre = input("Enter book genre: ").strip()
            copies = int(input("Enter number of copies: ").strip())
            add_book(book_id, title, author, genre, copies)
        elif choice == '5' and is_admin(user):
            book_id = int(input("Enter book ID to remove: ").strip())
            remove_book(book_id)
        elif choice == '6':
            send_overdue_notifications()
        elif choice == '7':
            book_id = int(input("Enter book ID to reserve: ").strip())
            reserve_book(user["user_id"], book_id)
        elif choice == '8':
            recommend_books(user["user_id"])
        elif choice == '9' and is_admin(user):
            most_borrowed_books()
            most_active_users()
        elif choice == '10':
            break
        else:
            print("Invalid option or insufficient permissions.")

# Start the user interaction console
user_interaction()

Welcome to the Library Management System
1. Register as a new user
2. Login to the system
Select an option: 2
=== Login ===
Enter your User ID: 302
Enter your password: adminpass
Welcome, Henry (Admin)!

--- Main Menu ---
1. Search Book
2. Borrow Book
3. Return Book
4. Add Book (Admin)
5. Remove Book (Admin)
6. Send Overdue Notifications
7. Reserve Book
8. Book Recommendations
9. View Library Statistics (Admin)
10. Exit
Select an option: 8
No borrow history available for recommendations.

--- Main Menu ---
1. Search Book
2. Borrow Book
3. Return Book
4. Add Book (Admin)
5. Remove Book (Admin)
6. Send Overdue Notifications
7. Reserve Book
8. Book Recommendations
9. View Library Statistics (Admin)
10. Exit
Select an option: 10
